In [4]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.sql.Row

import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.sql.Row


In [5]:
val training = spark.createDataFrame(Seq((1.0, Vectors.dense(0.0, 1.1, 0.1)),(0.0, Vectors.dense(2.0, 1.0, -1.0)),(0.0,Vectors.dense(2.0, 1.3, 1.0)),(1.0, Vectors.dense(0.0, 1.2, -0.5)))).toDF("label", "features")

training: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [6]:
training.show()

+-----+--------------+
|label|      features|
+-----+--------------+
|  1.0| [0.0,1.1,0.1]|
|  0.0|[2.0,1.0,-1.0]|
|  0.0| [2.0,1.3,1.0]|
|  1.0|[0.0,1.2,-0.5]|
+-----+--------------+



In [7]:
val lr = new LogisticRegression()

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_d42831fd086a


In [4]:
//设置迭代次数，正则化参数
lr.setMaxIter(10).setRegParam(0.01)

res0: lr.type = logreg_4327fda6a3d4


In [5]:
val model1 = lr.fit(training)

model1: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_4327fda6a3d4, numClasses=2, numFeatures=3


In [6]:
//model1.parent.extractParamMap查看模型参数
println("Model 1 was fit using parameters: " + model1.parent.extractParamMap)

Model 1 was fit using parameters: {
	logreg_4327fda6a3d4-aggregationDepth: 2,
	logreg_4327fda6a3d4-elasticNetParam: 0.0,
	logreg_4327fda6a3d4-family: auto,
	logreg_4327fda6a3d4-featuresCol: features,
	logreg_4327fda6a3d4-fitIntercept: true,
	logreg_4327fda6a3d4-labelCol: label,
	logreg_4327fda6a3d4-maxBlockSizeInMB: 0.0,
	logreg_4327fda6a3d4-maxIter: 10,
	logreg_4327fda6a3d4-predictionCol: prediction,
	logreg_4327fda6a3d4-probabilityCol: probability,
	logreg_4327fda6a3d4-rawPredictionCol: rawPrediction,
	logreg_4327fda6a3d4-regParam: 0.01,
	logreg_4327fda6a3d4-standardization: true,
	logreg_4327fda6a3d4-threshold: 0.5,
	logreg_4327fda6a3d4-tol: 1.0E-6
}


In [7]:
//paramMap设置模型参数 threshol为阈值
val paramMap = ParamMap(lr.maxIter -> 20).put(lr.maxIter, 30).put(lr.regParam -> 0.1, lr.threshold -> 0.55)

paramMap: org.apache.spark.ml.param.ParamMap =
{
	logreg_4327fda6a3d4-maxIter: 30,
	logreg_4327fda6a3d4-regParam: 0.1,
	logreg_4327fda6a3d4-threshold: 0.55
}


In [8]:
val paramMap2 = ParamMap(lr.probabilityCol -> "myProbability")

paramMap2: org.apache.spark.ml.param.ParamMap =
{
	logreg_4327fda6a3d4-probabilityCol: myProbability
}


In [9]:
val paramMapCombined = paramMap ++ paramMap2

paramMapCombined: org.apache.spark.ml.param.ParamMap =
{
	logreg_4327fda6a3d4-maxIter: 30,
	logreg_4327fda6a3d4-probabilityCol: myProbability,
	logreg_4327fda6a3d4-regParam: 0.1,
	logreg_4327fda6a3d4-threshold: 0.55
}


In [10]:
val model2 = lr.fit(training, paramMapCombined)

model2: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_4327fda6a3d4, numClasses=2, numFeatures=3


In [11]:
println("Model 2 was fit using parameters: " + model2.parent.extractParamMap)

Model 2 was fit using parameters: {
	logreg_4327fda6a3d4-aggregationDepth: 2,
	logreg_4327fda6a3d4-elasticNetParam: 0.0,
	logreg_4327fda6a3d4-family: auto,
	logreg_4327fda6a3d4-featuresCol: features,
	logreg_4327fda6a3d4-fitIntercept: true,
	logreg_4327fda6a3d4-labelCol: label,
	logreg_4327fda6a3d4-maxBlockSizeInMB: 0.0,
	logreg_4327fda6a3d4-maxIter: 30,
	logreg_4327fda6a3d4-predictionCol: prediction,
	logreg_4327fda6a3d4-probabilityCol: myProbability,
	logreg_4327fda6a3d4-rawPredictionCol: rawPrediction,
	logreg_4327fda6a3d4-regParam: 0.1,
	logreg_4327fda6a3d4-standardization: true,
	logreg_4327fda6a3d4-threshold: 0.55,
	logreg_4327fda6a3d4-tol: 1.0E-6
}


In [12]:
val test = spark.createDataFrame(Seq((1.0, Vectors.dense(-1.0, 1.5, 1.3)),(0.0, Vectors.dense(3.0, 2.0, -0.1)),(1.0,Vectors.dense(0.0, 2.2, -1.5)))).toDF("label", "features")

test: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [13]:
model2.transform(test).select("features", "label", "myProbability", "prediction").collect().foreach { case Row(features: Vector,label: Double, prob: Vector, prediction: Double) =>println(s"($features, $label) -> prob=$prob, prediction=$prediction")}

([-1.0,1.5,1.3], 1.0) -> prob=[0.05707304171034022,0.9429269582896598], prediction=1.0
([3.0,2.0,-0.1], 0.0) -> prob=[0.9238522311704104,0.07614776882958962], prediction=0.0
([0.0,2.2,-1.5], 1.0) -> prob=[0.10972776114779444,0.8902722388522055], prediction=1.0


In [14]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row


In [15]:
val training = spark.createDataFrame(Seq((0L, "a b c d e spark", 1.0),(1L, "b d", 0.0),(2L, "spark f g h", 1.0),(3L, "hadoop mapreduce",0.0))).toDF("id", "text", "label")

training: org.apache.spark.sql.DataFrame = [id: bigint, text: string ... 1 more field]


In [17]:
val tokenizer = new Tokenizer().setInputCol("text").setOutputCol("words")

val hashingTF = new HashingTF().setNumFeatures(1000).setInputCol(tokenizer.getOutputCol).setOutputCol("features")

val lr = new LogisticRegression().setMaxIter(10).setRegParam(0.001)

val pipeline = new Pipeline().setStages(Array(tokenizer, hashingTF, lr))

tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_8a86f499f1b9
hashingTF: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_325b4dc372b4, binary=false, numFeatures=1000
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_6b9c0ae13053
pipeline: org.apache.spark.ml.Pipeline = pipeline_13ebcbc5d346


In [18]:
val model = pipeline.fit(training)

model: org.apache.spark.ml.PipelineModel = pipeline_13ebcbc5d346


In [22]:
model.write.overwrite().save("spark-logistic-regression-model")

In [24]:
val test = spark.createDataFrame(Seq((4L, "spark i j k"),(5L, "l m n"),(6L, "spark hadoop spark"),(7L, "apache hadoop"))).toDF("id", "text")

test: org.apache.spark.sql.DataFrame = [id: bigint, text: string]


In [25]:
model.transform(test).select("id", "text", "probability", "prediction").collect().foreach { case Row(id: Long, text: String, prob: Vector,prediction: Double) =>println(s"($id, $text) --> prob=$prob, prediction=$prediction")}

(4, spark i j k) --> prob=[0.15964077387874104,0.8403592261212589], prediction=1.0
(5, l m n) --> prob=[0.8378325685476612,0.1621674314523388], prediction=0.0
(6, spark hadoop spark) --> prob=[0.06926633132976262,0.9307336686702374], prediction=1.0
(7, apache hadoop) --> prob=[0.9821575333444208,0.017842466655579203], prediction=0.0
